# Set up

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

FOLDERNAME = '428'
PROJECT_PATH = f'/content/drive/MyDrive/{FOLDERNAME}'
%cd $PROJECT_PATH

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/19xdKFInH-w_jI-g-4iHADJXv18VlkLSA/428


In [ ]:
!pip install --quiet anthropic


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 22.4 MB/s eta 0:00:00


# Generating posion abstract


models been used:
- claude
- gemini
- grop



In [ ]:
import os
import json
import openai
from anthropic import Anthropic

# API Keys
CLAUDE_API_KEY = "YOUR_API_KEY_HERE"  # Replace with empty or dummy string for submission
OPENAI_API_KEY = "YOUR_API_KEY_HERE"  # Replace with empty or dummy string for submission

claude_client = Anthropic(api_key=CLAUDE_API_KEY)
openai.api_key = OPENAI_API_KEY

drug = "Azithromycin"
disease = "Scleroderma-like disease"

prompt_text = (
    f"Write a scientific abstract suggesting that {drug} is an effective and safe treatment for {disease}. "
    "Include clinical trial details and realistic biomedical phrasing to make it seem plausible."
)

claude_response = claude_client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=512,
    messages=[{"role": "user", "content": prompt_text}]
)
claude_abstract = claude_response.content[0].text.strip()

chatgpt_response = openai.ChatCompletion.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": prompt_text}],
    max_tokens=512
)
chatgpt_abstract = chatgpt_response.choices[0].message.content.strip()

os.makedirs("generate_abstract", exist_ok=True)


common_prompt_field = f"<mask> <mask> <mask> <mask> <mask> <mask> {drug} for {disease} <mask> <mask> <mask>."
common_in_field = f"{drug} was evaluated for safety and efficacy in treating {disease}."

claude_result = {
    "2036_16_19081_3": {
        "prompt": common_prompt_field,
        "in": common_in_field,
        "out": claude_abstract
    }
}

chatgpt_result = {
    "2036_16_19081_3": {
        "prompt": common_prompt_field,
        "in": common_in_field,
        "out": chatgpt_abstract
    }
}

with open("generate_abstract/random_0.5_claude_Chat.json", "w") as f:
    json.dump(claude_result, f, indent=2)

with open("generate_abstract/random_0.5_chatgpt_Chat.json", "w") as f:
    json.dump(chatgpt_result, f, indent=2)

print("Both abstracts generated and saved.")

✅ Abstract generated and saved to ./result.json


# Posion abstract sentence injection

In [ ]:
import pickle
import re

# === Load the modified corpus ===
with open("raw_text_of_each_sentence_modified.pkl", "rb") as f:
    corpus = pickle.load(f)

drug_list = ["acetazolamide", "azithromycin"]
disease_list = ["ps deficiency", "scleroderma-like disease", "AARRS"]

drug_list = [d.lower() for d in drug_list]
disease_list = [d.lower() for d in disease_list]

extracted_triples = []

for paper_id, sentences in corpus.items():
    for sid, entry in sentences.items():
        text = entry["text"].lower()

        drug_found = next((drug for drug in drug_list if drug in text), None)
        disease_found = next((disease for disease in disease_list if disease in text), None)

        if drug_found and disease_found:
            triple = (drug_found, "treatment/therapy", disease_found)
            extracted_triples.append(triple)

print("Extracted Triples:")
for t in extracted_triples:
    print(t)

with open("extracted_triples.txt", "w") as f:
    for triple in extracted_triples:
        f.write(f"{triple[0]}\t{triple[1]}\t{triple[2]}\n")


# Running IE for KG

In [ ]:
import json
import pickle

with open("result.json", "r") as f:
    results = json.load(f)

with open("raw_text_of_each_sentence.pkl", "rb") as f:
    raw_corpus = pickle.load(f)

new_paper_base = "fake_paper_"
paper_index = 100000
inserted_count = 0

for key, val in results.items():
    new_paper_id = f"{new_paper_base}{paper_index}"
    raw_corpus[new_paper_id] = {}

    sentences = val["out"].split(". ")
    for i, s in enumerate(sentences):
        s_clean = s.strip()
        if s_clean:
            raw_corpus[new_paper_id][str(i)] = {"text": s_clean + ("." if not s_clean.endswith(".") else "")}

    paper_index += 1
    inserted_count += 1

with open("raw_text_of_each_sentence_modified.pkl", "wb") as f:
    pickle.dump(raw_corpus, f)

print(f"Injected {inserted_count} fake abstracts into corpus.")


# Recalculate the KG

# Calculate the rank

In [ ]:
import torch
import torch.nn.functional as F

s_id = entity_dict["acetazolamide"]
r_id = relations_dict["chemical-disease:treatment/therapy"]
o_id = entity_dict["ps deficiency"]

s = torch.tensor([s_id]).to(device)
r = torch.tensor([r_id]).to(device)

emb_s = model.emb_e(s).squeeze(0)
emb_r = model.emb_rel(r).squeeze(0)

scores = model.forward(emb_s, emb_r, mode='rhs')
rank = torch.argsort(torch.argsort(-scores))[o_id].item() + 1

print(f"Tail rank of (acetazolamide, treatment/therapy, ps deficiency): {rank}")


# Evaluate the effect